## IMDB top 1000 DB를 활용해
## 향후 '한국'의 '극장'에서 재개봉 했을 때 흥행할 영화 고르기

### team mates - 강현석, 오동준, 혁인

### 데이터 전처리 과정

In [6]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("data/IMDB top 1000.csv")

# 상위 5개 행 미리보기
print(df.head())


   Unnamed: 0                       Title                        Certificate  \
0       0                     1. The Shawshank Redemption (1994)         R     
1       1                                2. The Godfather (1972)         R     
2       2                              3. The Dark Knight (2008)     PG-13     
3       3                       4. The Godfather: Part II (1974)         R     
4       4      5. The Lord of the Rings: The Return of the Ki...     PG-13     

  Duration           Genre            Rate  Metascore  \
0  142 min                     Drama   9.3     80.0     
1  175 min              Crime, Drama   9.2    100.0     
2  152 min      Action, Crime, Drama   9.0     84.0     
3  202 min              Crime, Drama   9.0     90.0     
4  201 min  Action, Adventure, Drama   8.9     94.0     

                     Description                      \
0  Two imprisoned men bond over a number of years...   
1  The aging patriarch of an organized crime dyna...   
2  When 

In [7]:
import re

# 필요한 컬럼만 선택
df = df[["Title", "Genre", "Rate", "Info"]].copy()

# Votes와 Gross를 Info에서 추출
def extract_votes(info):
    match = re.search(r"Votes:\s([\d,]+)", str(info))
    return int(match.group(1).replace(",", "")) if match else None

def extract_gross(info):
    match = re.search(r"Gross:\s\$(\d+\.\d+)M", str(info))
    return float(match.group(1)) if match else None

df["Votes"] = df["Info"].apply(extract_votes)
df["Gross(M)"] = df["Info"].apply(extract_gross)

# 필요 없는 Info 컬럼 제거
df.drop(columns=["Info"], inplace=True)

# 장르 분리
df["Genre"] = df["Genre"].apply(lambda x: [g.strip() for g in str(x).split(",")])
df = df.explode("Genre").reset_index(drop=True)

# 결측치 제거 (평점, 매출, 장르 모두)
df = df.dropna(subset=["Rate", "Gross(M)", "Genre"])

# 타입 정리
df["Rate"] = df["Rate"].astype(float)
df["Gross(M)"] = df["Gross(M)"].astype(float)

# 전처리 결과 확인
print(df.info())
print(df.head(10))
print(" 전처리 완료! 총 행 개수:", len(df))

<class 'pandas.core.frame.DataFrame'>
Index: 2126 entries, 0 to 2505
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Title     2126 non-null   object 
 1   Genre     2126 non-null   object 
 2   Rate      2126 non-null   float64
 3   Votes     2126 non-null   int64  
 4   Gross(M)  2126 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 99.7+ KB
None
                        Title                          Genre     Rate  \
0                 1. The Shawshank Redemption (1994)      Drama   9.3   
1                            2. The Godfather (1972)      Crime   9.2   
2                            2. The Godfather (1972)      Drama   9.2   
3                          3. The Dark Knight (2008)     Action   9.0   
4                          3. The Dark Knight (2008)      Crime   9.0   
5                          3. The Dark Knight (2008)      Drama   9.0   
6                   4. The Godfather: Part II (

In [ ]:
# 위의 결과표를 주피터 노트북 상에서 예쁘게 뽑기 위해 진자2 설치
%pip install jinja2


   -------------------- ------------------- 1/2 [jinja2]
   ---------------------------------------- 2/2 [jinja2]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# 판다스 이용해서 표를 좀더 예쁘게 시각화 하기

# 보기 옵션 설정
pd.set_option("display.max_columns", None)
pd.set_option("display.colheader_justify", "center")

# 범위 고정
rate_min, rate_max   = df["Rate"].min(), df["Rate"].max()
gross_min, gross_max = df["Gross(M)"].min(), df["Gross(M)"].max()

styled_df = (
    df.head(15)  # 표시 건수는 필요에 맞게 조절
      .style
      # 숫자 포맷
      .format({
          "Rate": "{:.1f}",
          "Gross(M)": "{:,.2f}",
          "Votes": "{:,}"
      })
      # 스타일 지정
      # 헤더/캡션 가운데 정렬
      .set_table_styles([
          {"selector": "th", "props": [("text-align", "center")]},
          {"selector": "caption", "props": [
              ("text-align", "center"),
              ("font-weight", "600"),
              ("margin-bottom", "6px")
          ]},
      ])
      # 본문 셀 가운데 정렬 (Title/Genre 포함 전체)
      .set_properties(**{"text-align": "center"})
      # Rate 컬럼: 보기 편한 붉은 계열 음영
      .background_gradient(subset=["Rate"], cmap="Reds", vmin=rate_min, vmax=rate_max)
      # Gross(M) 컬럼: 초록 계열 음영
      .background_gradient(subset=["Gross(M)"], cmap="Greens", vmin=gross_min, vmax=gross_max)
      # Votes는 가로 막대 시각 보조 (원하면 주석 해제)
      # .bar(subset=["Votes"], color="#6c757d")
      .set_caption("IMDb 전처리 완료 데이터 (상위 15개 예시)")
)

styled_df  = styled_df.hide(axis="index")
styled_df

Title,Genre,Rate,Votes,Gross(M)
1. The Shawshank Redemption (1994),Drama,9.3,"2,295,987",28.34
2. The Godfather (1972),Crime,9.2,"1,584,782",134.97
2. The Godfather (1972),Drama,9.2,"1,584,782",134.97
3. The Dark Knight (2008),Action,9.0,"2,260,649",534.86
3. The Dark Knight (2008),Crime,9.0,"2,260,649",534.86
3. The Dark Knight (2008),Drama,9.0,"2,260,649",534.86
4. The Godfather: Part II (1974),Crime,9.0,"1,107,253",57.30
4. The Godfather: Part II (1974),Drama,9.0,"1,107,253",57.30
5. The Lord of the Rings: The Return of the King (2003),Action,8.9,"1,614,369",377.85
5. The Lord of the Rings: The Return of the King (2003),Adventure,8.9,"1,614,369",377.85


### 해당 영화들을 장르를 기준으로 분할하여 별도의 csv파일로 저장하기

In [20]:
import os, re
from pathlib import Path
import pandas as pd
from datetime import datetime

# 0) 저장 폴더 만들기
out_dir = Path("data/genres")
out_dir.mkdir(parents=True, exist_ok=True)

# 1) 장르 목록 (explode된 상태 기준)
genres = (
    df["Genre"]
    .dropna()
    .astype(str)
    .str.strip()
    .unique()
)

# 2) 파일명 안전하게 만드는 함수
def safe_name(name: str) -> str:
    name = name.lower().strip()
    name = name.replace("&", "and").replace("-", "_").replace(" ", "_")
    name = re.sub(r"[^a-z0-9_]", "", name)
    return name or "unknown"

# 3) 저장 로그 리스트 미리 생성
saved = []
skipped = []

# 4) 장르별 CSV 저장 (안전장치 포함)
for g in sorted(genres):
    sub = df[df["Genre"].str.strip() == g].copy()
    if sub.empty:
        continue
    sub = sub.sort_values(["Gross(M)", "Rate"], ascending=[False, False])
    
    fname = out_dir / f"{safe_name(g)}.csv"
    
    # 파일이 이미 존재하면 건너뜀
    if fname.exists():
        print(f"이미 존재: {fname.name} → 저장 건너뜀")
        skipped.append(fname.name)
        continue

    # 새로 저장
    sub.to_csv(fname, index=False, encoding="utf-8-sig")
    saved.append(fname.name)
    print(f"저장 완료: {fname.name} ({len(sub)}개)")

# 5) 요약표 저장 (덮어쓰기 대신 날짜 버전 생성)
summary = (
    df.groupby("Genre", dropna=True)
      .agg(
          n_titles=("Title", "count"),
          avg_rate=("Rate", "mean"),
          avg_gross_m=("Gross(M)", "mean"),
          total_gross_m=("Gross(M)", "sum")
      )
      .round({"avg_rate": 2, "avg_gross_m": 2, "total_gross_m": 2})
      .sort_values("n_titles", ascending=False)
      .reset_index()
)

summary_path = out_dir / "1_genre_summary.csv"
if summary_path.exists():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    backup_path = out_dir / f"genre_summary_{timestamp}.csv"
    summary.to_csv(backup_path, index=False, encoding="utf-8-sig")
    print(f"📂 기존 summary 존재 → 백업 저장: {backup_path.name}")
else:
    summary.to_csv(summary_path, index=False, encoding="utf-8-sig")
    print(f"✅ 요약표 저장 완료: {summary_path.name}")

# 6) 로그 출력
print("\n 저장 요약")
print(f"- 새로 저장된 파일: {len(saved)}개")
print(f"- 건너뛴 파일: {len(skipped)}개")


이미 존재: action.csv → 저장 건너뜀
이미 존재: adventure.csv → 저장 건너뜀
이미 존재: animation.csv → 저장 건너뜀
이미 존재: biography.csv → 저장 건너뜀
이미 존재: comedy.csv → 저장 건너뜀
이미 존재: crime.csv → 저장 건너뜀
이미 존재: drama.csv → 저장 건너뜀
이미 존재: family.csv → 저장 건너뜀
이미 존재: fantasy.csv → 저장 건너뜀
이미 존재: film_noir.csv → 저장 건너뜀
이미 존재: history.csv → 저장 건너뜀
이미 존재: horror.csv → 저장 건너뜀
이미 존재: music.csv → 저장 건너뜀
이미 존재: musical.csv → 저장 건너뜀
이미 존재: mystery.csv → 저장 건너뜀
이미 존재: romance.csv → 저장 건너뜀
이미 존재: sci_fi.csv → 저장 건너뜀
이미 존재: sport.csv → 저장 건너뜀
이미 존재: thriller.csv → 저장 건너뜀
이미 존재: war.csv → 저장 건너뜀
이미 존재: western.csv → 저장 건너뜀
✅ 요약표 저장 완료: 1_genre_summary.csv

 저장 요약
- 새로 저장된 파일: 0개
- 건너뛴 파일: 21개


### 범죄 장르의 영화 csv를 불러오고 매출, 평점 내림차순으로 상위영화 10개씩 뽑기

In [22]:
import pandas as pd
from pathlib import Path

# 1) 범죄 장르 CSV 경로
crime_path = Path("data/genres/crime.csv")

# 파일 존재 확인
if not crime_path.exists():
    raise FileNotFoundError(f"파일이 없습니다: {crime_path}. 장르 분할 저장을 먼저 실행해 주세요.")

# 2) CSV 불러오기
crime = pd.read_csv(crime_path)

# 필요한 컬럼만 보기 좋게
cols = ["Title", "Rate", "Votes", "Gross(M)"]
crime = crime[cols].copy()

# 숫자형 보장 (혹시 모를 문자열 섞임 대비)
crime["Rate"] = pd.to_numeric(crime["Rate"], errors="coerce")
crime["Votes"] = pd.to_numeric(crime["Votes"], errors="coerce")
crime["Gross(M)"] = pd.to_numeric(crime["Gross(M)"], errors="coerce")

# 결측치 제거
crime = crime.dropna(subset=["Rate", "Gross(M)"])

# 3) 매출 TOP 10 (동률 시: Votes, Rate 순으로 타이브레이크)
top_by_gross = (
    crime.sort_values(["Gross(M)", "Votes", "Rate"], ascending=[False, False, False])
         .head(10)
         .reset_index(drop=True)
)

# 4) 평점 TOP 10 (동률 시: Votes, Gross 순으로 타이브레이크)
top_by_rate = (
    crime.sort_values(["Rate", "Votes", "Gross(M)"], ascending=[False, False, False])
         .head(10)
         .reset_index(drop=True)
)

# 5) 결과 확인 (노트북에서 보기)
display(top_by_gross.style.format({"Rate":"{:.1f}", "Gross(M)":"{:,.2f}", "Votes":"{:,.0f}"}).set_caption("💰 범죄 장르 — 매출 TOP 10"))
display(top_by_rate.style.format({"Rate":"{:.1f}", "Gross(M)":"{:,.2f}", "Votes":"{:,.0f}"}).set_caption("⭐ 범죄 장르 — 평점 TOP 10"))

# # 6) (선택) 파일로 저장
# out_dir = Path("data/genres/top")
# out_dir.mkdir(parents=True, exist_ok=True)
# top_by_gross.to_csv(out_dir / "crime_top_by_gross.csv", index=False, encoding="utf-8-sig")
# top_by_rate.to_csv(out_dir / "crime_top_by_rate.csv", index=False, encoding="utf-8-sig")

# print(" 완료: crime_top_by_gross.csv / crime_top_by_rate.csv 저장")


,Title,Rate,Votes,Gross(M)
0,3. The Dark Knight (2008),9.0,"2,260,649",534.86
1,33. Joker (2019),8.5,"882,903",335.45
2,242. Catch Me If You Can (2002),8.1,"808,593",164.62
3,113. The Sting (1973),8.3,"237,865",159.60
4,25. The Green Mile (1999),8.6,"1,122,263",136.80
5,2. The Godfather (1972),9.2,"1,584,782",134.97
6,37. The Departed (2006),8.5,"1,167,751",132.38
7,28. The Silence of the Lambs (1991),8.6,"1,244,794",130.74
8,147. The Wolf of Wall Street (2013),8.2,"1,152,702",116.90
9,6. Pulp Fiction (1994),8.9,"1,792,919",107.93


,Title,Rate,Votes,Gross(M)
0,2. The Godfather (1972),9.2,"1,584,782",134.97
1,3. The Dark Knight (2008),9.0,"2,260,649",534.86
2,4. The Godfather: Part II (1974),9.0,"1,107,253",57.30
3,6. Pulp Fiction (1994),8.9,"1,792,919",107.93
4,8. 12 Angry Men (1957),8.9,"674,629",4.36
5,17. Goodfellas (1990),8.7,"999,623",46.84
6,27. Se7en (1995),8.6,"1,414,169",100.13
7,28. The Silence of the Lambs (1991),8.6,"1,244,794",130.74
8,25. The Green Mile (1999),8.6,"1,122,263",136.80
9,22. City of God (2002),8.6,"689,676",7.56
